In [1]:
import geopandas
from openai import OpenAI

import os
import wget
import zipfile

from dotenv import load_dotenv

from hackathonopenai.hydrological import HydrologicalAssistant

%load_ext autoreload
%autoreload 2

In [2]:
def download_hidro_data(
        url_path: str = "https://www.geoportal.cl/geoportal/catalog/download/5c781642-c351-3a15-bbb3-1e850ce3e52d",
        output_directory_base: str = "../data/hidro",
        raw_data: str = '../data/raw',
        name: str = "data_hidro"
) -> str:
    """
    Download the hidro data from the url_path and save it in the output_directory_base
    """
    if not os.path.exists(output_directory_base):
        os.makedirs(output_directory_base, exist_ok=True)

    if not os.path.exists(raw_data):
        os.makedirs(raw_data, exist_ok=True)

    # Download the data
    output_directory = wget.download(url_path, out=raw_data)
    # Unzip the data
    with zipfile.ZipFile(output_directory, 'r') as zip_ref:
        zip_ref.extractall(output_directory_base)

    for file in os.listdir(output_directory_base):
        os.rename(
            os.path.join(output_directory_base, file), 
            os.path.join(output_directory_base, name + os.path.splitext(file)[1])
        )
    return output_directory_base

In [3]:
load_dotenv("../.env")
name = "data_hidro"
output_directory_base = download_hidro_data(name=name)

# Load files
df = geopandas.read_file(
    os.path.join(output_directory_base, name + ".shp")
)
df_demo = geopandas.read_file(
    "../data/demo_data/doc.kml"
)

In [5]:
from shapely.ops import unary_union

single_polygon = unary_union(df_demo.geometry)
single_polygon_df = geopandas.GeoDataFrame(geometry=[single_polygon], crs=df_demo.crs)

In [6]:
df_kms = df.to_crs("EPSG:32633")
demo_geometry_kms = single_polygon_df.to_crs("EPSG:32633")

In [7]:
df_kms = df_kms[~df_kms['nom_cuen'].astype(str).str.startswith('Isla')]

In [11]:
hydrological_assistant = HydrologicalAssistant(df_kms, OpenAI())

In [12]:
response = hydrological_assistant.evaluate_project(demo_geometry_kms)

/opt/miniconda3/envs/hackathon/lib/python3.12/site-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


Numero de cuerpos de agua cercanos:  3


In [14]:
response

{'summary': 'El proyecto fotovoltaico presenta impactos críticos debido a la intersección con cuerpos de agua, lo que podría resultar en la negativa del proyecto. Se recomienda modificar el polígono del proyecto para evitar la intersección con los cuerpos de agua o incurrir en permisos ambientales sectoriales.',
 'detailed_impact_assessment': {'intersecting_water_bodies': [{'cod_cuen': '055',
    'cod_subc': '0552',
    'cod_ssubc': '05520',
    'nom_cuen': 'Costeras entre Aconcagua y Maipo',
    'nom_subc': 'E. Casablanca y E.  San Jeronimo (Incl.)',
    'nom_ssubc': 'Costeras entre Estero Casablanca y Estero San Jeronimo (Incl.)',
    'tipo_bcn': 'Quebrada',
    'distance_in_mts': 0.0},
   {'cod_cuen': '055',
    'cod_subc': '0552',
    'cod_ssubc': '05520',
    'nom_cuen': 'Costeras entre Aconcagua y Maipo',
    'nom_subc': 'E. Casablanca y E.  San Jeronimo (Incl.)',
    'nom_ssubc': 'Costeras entre Estero Casablanca y Estero San Jeronimo (Incl.)',
    'tipo_bcn': 'Quebrada',
    'd